This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/development/mlops_api).

## Wallaroo MLOps API Tutorial

The Wallaroo MLOps API allows organizations to submit requests to their Wallaroo instance to perform such actions as:

* Create a new user and invite them to the instance.
* Create workspaces and list their configuration details.
* Upload a model.
* Deploy and undeploy a pipeline.

The following examples will show how to submit queries to the Wallaroo MLOps API and the types of responses returned.

### References

The following references are available for more information about Wallaroo and the Wallaroo MLOps API:

* [Wallaroo Documentation Site](https://docs.wallaroo.ai):  The Wallaroo Documentation Site
* Wallaroo MLOps API Documentation from a Wallaroo instance:  A Swagger UI based documentation is available from your Wallaroo instance at `https://{Wallaroo Prefix}.api.{Wallaroo Suffix}/v1/api/docs`.  For example, if the Wallaroo Instance is YOUR SUFFIX with the prefix `{lovely-rhino-5555}`, then the Wallaroo MLOps API Documentation would be available at `https://lovely-rhino-5555.api.example.wallaroo.ai/v1/api/docs`.  For another example, a Wallaroo Enterprise users who do not use a prefix and has the suffix `wallaroo.example.wallaroo.ai`, the the Wallaroo MLOps API Documentation would be available at `https://api.wallaroo.example.wallaroo.ai/v1/api/docs`.  For more information, see the [Wallaroo Documentation Site](https://docs.wallaroo.ai).

**IMPORTANT NOTE**:  The Wallaroo MLOps API is provided as an early access features.  Future iterations may adjust the methods and returns to provide a better user experience.  Please refer to this guide for updates.

## OpenAPI Steps

The following demonstrates how to use each command in the Wallaroo MLOps API, and can be modified as best fits your organization's needs.

### Import Libraries

For the examples, the Python `requests` library will be used to make the REST HTTP(S) connections.  `import uuid` will be used to create workspaces, pipelines, assays and other items uniquely so we don't go clobbering over existing items.

In [5]:
# Requires requests and requests-toolbelt with either:
# pip install requests-toolbelt
# conda install -c conda-forge requests-toolbelt

import requests
from requests.auth import HTTPBasicAuth

import uuid

## Retrieve Credentials

### Through Keycloak

Wallaroo comes pre-installed with a confidential OpenID Connect client.  The default client is `api-client`, but other clients may be created and configured.

As it is a confidential client, api-client requires its secret to be supplied when requesting a token. Administrators may obtain their API client credentials from Keycloak from the Keycloak Service URL as listed above and the prefix `/auth/admin/master/console/#/realms/master/clients`.

For example, if the YOUR SUFFIX instance DNS address is `https://magical-rhino-5555.example.wallaroo.ai`, then the direct path to the Keycloak API client credentials would be:

`https://magical-rhino-5555.keycloak.example.wallaroo.ai/auth/admin/master/console/#/realms/master/clients`

Then select the client, in this case **api-client**, then **Credentials**.

![Wallaroo Keycloak Service](./images/wallaroo-developer-guides/wallaroo-api/wallaroo-api-keycloak-service.png)

![Wallaroo Components](./images/wallaroo-developer-guides/wallaroo-api/wallaroo-api-keycloak-credentials.png)

By default, tokens issued for api-client are valid for up to 60 minutes. Refresh tokens are supported.

### Through the Wallaroo SDK

The API token can be retrieved using the [Wallaroo SDK](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-install-guide) through the `wallaroo.client.mlops()` command. In the following example, the token will be retrieved and stored to the variable `TOKEN`:

```python
connection =wl.mlops().__dict__
TOKEN = connection['token']
print(TOKEN)
'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJnTHBSY1B6QkhjQ1k1RTFHTVZoTlQtelI0VDY2YUM0QWh2eXVORmpVOTBjIn0.eyJleHAiOjE2NzEwMzMzMzUsImlhdCI6MTY3MTAzMzI3NSwiYXV0aF90aW1lIjoxNjcxMDMyODgyLCJqdGkiOiJiNDk3YmM3Yy1kMTc5LTRhYWQtODdmZC0yZGJiYTBlZDI4ZDYiLCJpc3MiOiJodHRwczovL21hZ2ljYWwtYmVhci0zNzgyLmtleWNsb2FrLndhbGxhcm9vLmNvbW11bml0eS9hdXRoL3JlYWxtcy9tYXN0ZXIiLCJhdWQiOlsibWFzdGVyLXJlYWxtIiwiYWNjb3VudCJdLCJzdWIiOiJmMWYzMmJkZi05YmQ5LTQ1OTUtYTUzMS1hY2E1Nzc4Y2VhZjAiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzZGstY2xpZW50Iiwic2Vzc2lvbl9zdGF0ZSI6IjYzYzNiZjYwLTNmNjMtNDBjNC05NmI1LWNiYTk4ZjZhOGNmNyIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiY3JlYXRlLXJlYWxtIiwiZGVmYXVsdC1yb2xlcy1tYXN0ZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJtYXN0ZXItcmVhbG0iOnsicm9sZXMiOlsidmlldy1pZGVudGl0eS1wcm92aWRlcnMiLCJ2aWV3LXJlYWxtIiwibWFuYWdlLWlkZW50aXR5LXByb3ZpZGVycyIsImltcGVyc29uYXRpb24iLCJjcmVhdGUtY2xpZW50IiwibWFuYWdlLXVzZXJzIiwicXVlcnktcmVhbG1zIiwidmlldy1hdXRob3JpemF0aW9uIiwicXVlcnktY2xpZW50cyIsInF1ZXJ5LXVzZXJzIiwibWFuYWdlLWV2ZW50cyIsIm1hbmFnZS1yZWFsbSIsInZpZXctZXZlbnRzIiwidmlldy11c2VycyIsInZpZXctY2xpZW50cyIsIm1hbmFnZS1hdXRob3JpemF0aW9uIiwibWFuYWdlLWNsaWVudHMiLCJxdWVyeS1ncm91cHMiXX0sImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoiZW1haWwgcHJvZmlsZSIsInNpZCI6IjYzYzNiZjYwLTNmNjMtNDBjNC05NmI1LWNiYTk4ZjZhOGNmNyIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJodHRwczovL2hhc3VyYS5pby9qd3QvY2xhaW1zIjp7IngtaGFzdXJhLXVzZXItaWQiOiJmMWYzMmJkZi05YmQ5LTQ1OTUtYTUzMS1hY2E1Nzc4Y2VhZjAiLCJ4LWhhc3VyYS1kZWZhdWx0LXJvbGUiOiJ1c2VyIiwieC1oYXN1cmEtYWxsb3dlZC1yb2xlcyI6WyJ1c2VyIl0sIngtaGFzdXJhLXVzZXItZ3JvdXBzIjoie30ifSwicHJlZmVycmVkX3VzZXJuYW1lIjoiam9obi5oYW5zYXJpY2tAd2FsbGFyb28uYWkiLCJlbWFpbCI6ImpvaG4uaGFuc2FyaWNrQHdhbGxhcm9vLmFpIn0.EEt9UK1jxvO1DYg_hiy1ne4s9iK8mJtqbVfE7MPQfMRYhzXqDU4gFpP3Nwzlo0iW9fSLDiCxPg303Rz-l4it3oPFu5SaS1S8pQpqvtMAJqy8V_CNPp5H5ggQFYm4Z50aAPdPzOOOkVQOZUhupRsEeUERvK1-eFqtG1bb-IUV6DpQO_XaRVcQbIVubFi48C0_im5Tb3i4WFCNA_1pRrEBKFbZLWgzSCu8fglBQ27mODqfmRQVbTeXLjxsQX5O8meErSfibEGmsJKQytGCJ3NYdnXfal3YhWEqp6A4dG0tkoRW1eD-aKBpsHf9nKKzxcSsjeXDQF6iQAONCGmC40oqHQ'
```

### Set Variables

The following variables are used for the example and should be modified to fit your organization.

In [6]:
## Variables

URLPREFIX='YOUR PREFIX'
URLSUFFIX='YOUR SUFFIX'
SECRET="YOUR SECRET"
TOKENURL=f'https://{URLPREFIX}.keycloak.{URLSUFFIX}/auth/realms/master/protocol/openid-connect/token'
CLIENT="api-client"
USERNAME="YOUR USERNAME"
PASSWORD="YOUR PASSWORD"
APIURL=f"https://{URLPREFIX}.api.{URLSUFFIX}/v1/api"
newUser="NEW USER EMAIL"
newPassword="NEW USER PASSWORD"

The following is an output of the `TOKENURL` variable to verify it matches your Wallaroo instance's Keycloak API client credentials URL.

In [7]:
TOKENURL

'https://YOUR PREFIX.keycloak.YOUR SUFFIX/auth/realms/master/protocol/openid-connect/token'

### API Example Methods

The following methods are used to retrieve the MLOPs API Token from the Wallaroo instance's Keycloak service, and submit MLOps API requests through the Wallaroo instance's MLOps API.

MLOps API requests are always `POST`, and are either submitted as `'Content-Type':'application/json'` or as a multipart submission including a file.

In [8]:
def get_jwt_token(url, client, secret, username, password):
    auth = HTTPBasicAuth(client, secret)
    data = {
        'grant_type': 'password',
        'username': username,
        'password': password
    }
    response = requests.post(url, auth=auth, data=data, verify=True)
    return response.json()['access_token']


# This can either submit a plain POST request ('Content-Type':'application/json'), or with a file.

def get_wallaroo_response(url, api_request, token, data, files=None, contentType='application/json', params=None):
    apiUrl=f"{url}{api_request}"
    if files is None:
        # Regular POST request
        headers= {
            'Authorization': 'Bearer ' + token,
            'Content-Type':contentType
        }
        response = requests.post(apiUrl, json=data, headers=headers, verify=True)
    elif contentType == 'application/octet-stream':
        # Post request as octet-stream
        headers= {
            'Authorization': 'Bearer ' + token,
            'Content-Type':contentType
        }
        response = requests.post(apiUrl, data=files, headers=headers, params=params)
        #response = requests.post(apiUrl, data=data, headers=headers, files=files, verify=True)
    else:
        # POST request with file
        headers= {
            'Authorization': 'Bearer ' + token
        }
        response = requests.post(apiUrl, data=data, headers=headers, files=files, verify=True)
    return response.json()


### Retrieve MLOps API Token

To retrieve an API token for a specific user with the Client Secret, request the token from the Wallaroo instance using the client secret and provide the following:

* Token Request URL: The Keycloak token retrieval URL.
* OpenID Connect client name: The name of the OpenID Connect client.
* OpenID Connect client Secret:  The secret for the OpenID Connect client.
* UserName:  The username of the Wallaroo instance user, usually their email address.
* Password:  The password of the Wallaroo instance user.

The following sample uses the variables set above to request the token, then displays it.

In [9]:
TOKEN=get_jwt_token(TOKENURL, CLIENT, SECRET, USERNAME, PASSWORD)

In [10]:
TOKEN

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJWalFITFhMMThub3BXNWVHM2hMOVJ5MDZ1SFVWMko1dHREUkVxSGtBT2VzIn0.eyJleHAiOjE2NzE1NjY2MzYsImlhdCI6MTY3MTU2MzAzNiwianRpIjoiNTdjZTkzMmYtYjZmYS00MGFkLThhNzMtMWMyMzE4OTAwNWJjIiwiaXNzIjoiaHR0cHM6Ly9zcXVpc2h5LXdhbGxhcm9vLTYxODcua2V5Y2xvYWsud2FsbGFyb28uZGV2L2F1dGgvcmVhbG1zL21hc3RlciIsImF1ZCI6WyJtYXN0ZXItcmVhbG0iLCJhY2NvdW50Il0sInN1YiI6IjAxYTc5N2Y5LTEzNTctNDUwNi1hNGQyLThhYjljNDY4MTEwMyIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFwaS1jbGllbnQiLCJzZXNzaW9uX3N0YXRlIjoiZDZlYzdkODUtNWExMC00ZDJjLWE3NTUtYjI4YzI1NjJlNDYyIiwiYWNyIjoiMSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJjcmVhdGUtcmVhbG0iLCJkZWZhdWx0LXJvbGVzLW1hc3RlciIsIm9mZmxpbmVfYWNjZXNzIiwiYWRtaW4iLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7Im1hc3Rlci1yZWFsbSI6eyJyb2xlcyI6WyJ2aWV3LWlkZW50aXR5LXByb3ZpZGVycyIsInZpZXctcmVhbG0iLCJtYW5hZ2UtaWRlbnRpdHktcHJvdmlkZXJzIiwiaW1wZXJzb25hdGlvbiIsImNyZWF0ZS1jbGllbnQiLCJtYW5hZ2UtdXNlcnMiLCJxdWVyeS1yZWFsbXMiLCJ2aWV3LWF1dGhvcml6YXRpb24iLCJ

## Users

### Get Users

Users can be retrieved either by their Keycloak user id, or return all users if an empty set `{}` is submitted.

* **Parameters**
  * `{}`: Empty set, returns all users.
  * **user_ids** *Array[Keycloak user ids]*: An array of Keycloak user ids, typically in UUID format.

Example:  The first example will submit an empty set `{}` to return all users, then submit the first user's user id and request only that user's details.

In [33]:
# Get all users

apiRequest = "/users/query"
data = {
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'users': {'5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f': {'access': {'impersonate': True,
    'manageGroupMembership': True,
    'manage': True,
    'mapRoles': True,
    'view': True},
   'createdTimestamp': 1669221287375,
   'disableableCredentialTypes': [],
   'email': 'john.hansarick@wallaroo.ai',
   'emailVerified': True,
   'enabled': True,
   'id': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'notBefore': 0,
   'requiredActions': [],
   'username': 'john.hansarick@wallaroo.ai'},
  '941937b3-7dc8-4abe-8bb1-bd23c816421e': {'access': {'view': True,
    'manage': True,
    'manageGroupMembership': True,
    'mapRoles': True,
    'impersonate': True},
   'createdTimestamp': 1669221214282,
   'disableableCredentialTypes': [],
   'emailVerified': False,
   'enabled': True,
   'id': '941937b3-7dc8-4abe-8bb1-bd23c816421e',
   'notBefore': 0,
   'requiredActions': [],
   'username': 'admin'},
  'da7c2f4c-822e-49eb-93d7-a4b90af9b4ca': {'access': {'mapRoles': True,
    'impersonate': True,
    'man

In [34]:
# Get first user Keycloak id
firstUserKeycloak = list(response['users'])[0]

apiRequest = "/users/query"
data = {
  "user_ids": [
    firstUserKeycloak
  ]
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'users': {'5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f': {'access': {'view': True,
    'manage': True,
    'manageGroupMembership': True,
    'mapRoles': True,
    'impersonate': True},
   'createdTimestamp': 1669221287375,
   'disableableCredentialTypes': [],
   'email': 'john.hansarick@wallaroo.ai',
   'emailVerified': True,
   'enabled': True,
   'id': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'notBefore': 0,
   'requiredActions': [],
   'username': 'john.hansarick@wallaroo.ai'}}}

### Invite Users

**IMPORTANT NOTE**:  This command is for YOUR SUFFIX only.  For more details on user management, see [Wallaroo User Management](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-user-management/).

Users can be invited through `/users/invite`.  When using YOUR SUFFIX, this will send an invitation email to the email address listed.  Note that the user must not already be a member of the Wallaroo instance, and email addresses must be unique.  If the email address is already in use for another user, the request will generate an error.

* **Parameters**
  * **email** *(REQUIRED string): The email address of the new user to invite.
  * **password** *(OPTIONAL string)*: The assigned password of the new user to invite.  If not provided, the Wallaroo instance will provide the new user a temporary password that must be changed upon initial login.

Example:  In this example, a new user will be invited to the Wallaroo instance and assigned a password.

In [ ]:
# invite users
apiRequest = "/users/invite"
data = {
    "email": newUser,
    "password":newPassword
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

### Deactivate User

Users can be deactivated so they can not login to their Wallaroo instance.  Deactivated users do not count against the Wallaroo license count.

* **Parameters**
  * **email** (*REQUIRED string*):  The email address of the user to deactivate.

Example:  In this example, the `newUser` will be deactivated.

In [36]:
# Deactivate users

apiRequest = "/users/deactivate"

data = {
    "email": newUser
}

In [37]:
response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{}

### Activate User

A deactivated user can be reactivated to allow them access to their Wallaroo instance.  Activated users count against the Wallaroo license count.

* **Parameters**
  * **email** (*REQUIRED string*):  The email address of the user to activate.

Example:  In this example, the `newUser` will be activated.

In [38]:
# Activate users

apiRequest = "/users/activate"

data = {
    "email": newUser
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{}

## Workspaces

### List Workspaces

List the workspaces for a specific user.

* **Parameters**
  * **user_id** - (*OPTIONAL string*): The Keycloak ID.
  
Example:  In this example, the workspaces for the a specific user will be displayed, then workspaces for all users will be displayed.

In [39]:
# List workspaces by user id

apiRequest = "/workspaces/list"

data = {
    "user_id":firstUserKeycloak
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'workspaces': [{'id': 1,
   'name': 'john.hansarick@wallaroo.ai - Default Workspace',
   'created_at': '2022-11-23T16:34:47.914362+00:00',
   'created_by': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'archived': False,
   'models': [],
   'pipelines': []},
  {'id': 2,
   'name': 'alohaworkspace',
   'created_at': '2022-11-23T16:44:28.782225+00:00',
   'created_by': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'archived': False,
   'models': [1],
   'pipelines': [1]},
  {'id': 4,
   'name': 'testapiworkspace-cdf86c3c-8c9a-4bf4-865d-fe0ec00fad7c',
   'created_at': '2022-11-28T16:48:29.622794+00:00',
   'created_by': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'archived': False,
   'models': [2, 3, 5, 4, 6, 7, 8, 9],
   'pipelines': []}]}

In [40]:
# List workspaces

apiRequest = "/workspaces/list"

data = {
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'workspaces': [{'id': 1,
   'name': 'john.hansarick@wallaroo.ai - Default Workspace',
   'created_at': '2022-11-23T16:34:47.914362+00:00',
   'created_by': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'archived': False,
   'models': [],
   'pipelines': []},
  {'id': 2,
   'name': 'alohaworkspace',
   'created_at': '2022-11-23T16:44:28.782225+00:00',
   'created_by': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'archived': False,
   'models': [1],
   'pipelines': [1]},
  {'id': 4,
   'name': 'testapiworkspace-cdf86c3c-8c9a-4bf4-865d-fe0ec00fad7c',
   'created_at': '2022-11-28T16:48:29.622794+00:00',
   'created_by': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'archived': False,
   'models': [2, 3, 5, 4, 6, 7, 8, 9],
   'pipelines': []}]}

### Create Workspace

A new workspace will be created in the Wallaroo instance.  Upon creating, the workspace owner will be assigned as the user making the MLOps API request.

* **Parameters**:
  * **workspace_name** - (*REQUIRED string*):  The name of the new workspace.
* **Returns**:
  * **workspace_id** - (*int*):  The ID of the new workspace.
  
Example:  In this example, a workspace with the name `testapiworkspace-` with a randomly generated UUID will be created, and the newly created workspace's `workspace_id` saved as the variable `exampleWorkspaceId` for use in other code examples.  After the request is complete, the [List Workspaces](#list-workspaces) command will be issued to demonstrate the new workspace has been created.

In [24]:
# Create workspace

apiRequest = "/workspaces/create"

exampleWorkspaceName = f"testapiworkspace-{uuid.uuid4()}"
data = {
  "workspace_name": exampleWorkspaceName
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
# Stored for future examples
exampleWorkspaceId = response['workspace_id']
response

{'workspace_id': 618489}

In [25]:
# List workspaces

apiRequest = "/workspaces/list"

data = {
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'workspaces': [{'id': 15,
   'name': 'john.hansarick@wallaroo.ai - Default Workspace',
   'created_at': '2022-12-16T20:23:23.150058+00:00',
   'created_by': '01a797f9-1357-4506-a4d2-8ab9c4681103',
   'archived': False,
   'models': [],
   'pipelines': []},
  {'id': 28,
   'name': 'alohaworkspace',
   'created_at': '2022-12-16T21:00:01.614796+00:00',
   'created_by': '01a797f9-1357-4506-a4d2-8ab9c4681103',
   'archived': False,
   'models': [2],
   'pipelines': [4]},
  {'id': 29,
   'name': 'abtestworkspace',
   'created_at': '2022-12-16T21:03:08.785538+00:00',
   'created_by': '01a797f9-1357-4506-a4d2-8ab9c4681103',
   'archived': False,
   'models': [3, 5, 4, 6],
   'pipelines': [6]},
  {'id': 618487,
   'name': 'sdkquickworkspace',
   'created_at': '2022-12-20T15:56:22.088161+00:00',
   'created_by': '01a797f9-1357-4506-a4d2-8ab9c4681103',
   'archived': False,
   'models': [48],
   'pipelines': [76]},
  {'id': 618489,
   'name': 'testapiworkspace-e9e386a7-8146-4ead-b4c6-a2580af7008

### Add User to Workspace

Existing users of the Wallaroo instance can be added to an existing workspace.

* **Parameters**
  * **email** - (*REQUIRED string*):  The email address of the user to add to the workspace.
  * **workspace_id** - (*REQUIRED int*): The id of the workspace.
  
Example:  The following example adds the user created in Invite Users request to the workspace created in the [Create Workspace](#create-workspace) request.

In [43]:
# Add existing user to existing workspace

apiRequest = "/workspaces/add_user"

data = {
  "email":newUser,
  "workspace_id": exampleWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{}

### List Users in a Workspace

Lists the users who are either owners or collaborators of a workspace.

* **Parameters**
  * **workspace_id** - (*REQUIRED int*): The id of the workspace.
* **Returns**
  * **user_id**:  The user's identification.
  * **user_type**:  The user's workspace type (owner, co-owner, etc).
  
Example:  The following example will list all users part of the workspace created in the [Create Workspace](#create-workspace) request.

In [44]:
# List users in a workspace

apiRequest = "/workspaces/list_users"

data = {
  "workspace_id": exampleWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'users': [{'user_id': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'user_type': 'OWNER'},
  {'user_id': 'da7c2f4c-822e-49eb-93d7-a4b90af9b4ca',
   'user_type': 'COLLABORATOR'}]}

### Remove User from a Workspace

Removes the user from the given workspace.  In this request, either the user's Keycloak ID is required **OR** the user's email address is required.

* **Parameters**
  * **workspace_id** - (*REQUIRED int*): The id of the workspace.
  * **user_id** - (*string*): The Keycloak ID of the user.  If `email` is not provided, then this parameter is **REQUIRED**.
  * **email** - (*string*): The user's email address.  If `user_id` is not provided, then this parameter is **REQUIRED**.
* **Returns**
  * **user_id**:  The user's identification.
  * **user_type**:  The user's workspace type (owner, co-owner, etc).
  
Example:  The following example will remove the `newUser` from workspace created in the [Create Workspace](#create-workspace) request.  Then the users for that workspace will be listed to verify `newUser` has been removed.

In [45]:
# Remove existing user from an existing workspace

apiRequest = "/workspaces/remove_user"

data = {
  "email":newUser,
  "workspace_id": exampleWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'affected_rows': 1}

In [46]:
# List users in a workspace

apiRequest = "/workspaces/list_users"

data = {
  "workspace_id": exampleWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'users': [{'user_id': '5e9c9a2b-7a7f-454a-b8e7-91e3c2d86c9f',
   'user_type': 'OWNER'}]}

## Models

### Upload Model to Workspace

Uploads a ML Model to a Wallaroo workspace via POST with `Content-Type: multipart/form-data`.

* **Parameters**
  * **name** - (*REQUIRED string*): Name of the model
  * **visibility** - (*OPTIONAL string*): The visibility of the model as either `public` or `private`.
  * **workspace_id** - (*REQUIRED int*): The numerical id of the workspace to upload the model to.
  
Example:  This example will upload the sample file `ccfraud.onnx` to the workspace created in the [Create Workspace](#create-workspace) step as `apitestmodel`.  The model name will be saved as `exampleModelName` for use in other examples.  The id of the uploaded model will be saved as `exampleModelId` for use in later examples.

In [26]:
# upload model - uses multiform data through a Python `request`

apiRequest = "/models/upload"

exampleModelName = f"apitestmodel-{uuid.uuid4()}"

data = {
    "name":exampleModelName,
    "visibility":"public",
    "workspace_id": exampleWorkspaceId
}

files = {
    "file": ('ccfraud.onnx', open('./models/ccfraud.onnx','rb'))
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data, files)
response

{'insert_models': {'returning': [{'models': [{'id': 68}]}]}}

In [48]:
exampleModelId=response['insert_models']['returning'][0]['models'][0]['id']
exampleModelId

10

### Stream Upload Model to Workspace

Streams a potentially large ML Model to a Wallaroo workspace via POST with `Content-Type: multipart/form-data`.

* **Parameters**
  * **name** - (*REQUIRED string*): Name of the model
  * **filename** - (*REQUIRED string*): Name of the file being uploaded.
  * **visibility** - (*OPTIONAL string*): The visibility of the model as either `public` or `private`.
  * **workspace_id** - (*REQUIRED int*): The numerical id of the workspace to upload the model to.
  
Example:  This example will upload the sample file `ccfraud.onnx` to the workspace created in the [Create Workspace](#create-workspace) step as `apitestmodel`.

In [49]:
# stream upload model - next test is adding arbitrary chunks to the stream

apiRequest = "/models/upload_stream"
exampleModelName = f"apitestmodel-{uuid.uuid4()}"
filename = 'streamfile.onnx'

data = {
    "name":exampleModelName,
    "filename": 'streamfile.onnx',
    "visibility":"public",
    "workspace_id": exampleWorkspaceId
}

contentType='application/octet-stream'

file = open('./models/ccfraud.onnx','rb')

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data=None, files=file, contentType='application/octet-stream', params=data)
response

{'insert_models': {'returning': [{'models': [{'id': 11}]}]}}

### List Models in Workspace

Returns a list of models added to a specific workspace.

* **Parameters**
  * **workspace_id** - (*REQUIRED int*): The workspace id to list.
  
Example:  Display the models for the workspace used in the Upload Model to Workspace step.  The model id and model name will be saved as `exampleModelId` and `exampleModelName` variables for other examples.

In [27]:
# List models in a workspace

apiRequest = "/models/list"

data = {
  "workspace_id": exampleWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'models': [{'id': 68,
   'name': 'apitestmodel-dfa7e9fd-df72-4b28-93f6-3d147c9f962f',
   'owner_id': '""',
   'created_at': '2022-12-20T19:34:47.014072+00:00',
   'updated_at': '2022-12-20T19:34:47.014072+00:00'}]}

In [29]:
exampleModelId = response['models'][0]['id']
exampleModelName = response['models'][0]['name']

### Get Model Details By ID

Returns the model details by the specific model id.

* **Parameters**
  * **workspace_id** - (*REQUIRED int*): The workspace id to list.
* **Returns**
  * **id** - (*int*):  Numerical id of the model.
  * **owner_id** - (*string*): Id of the owner of the model.
  * **workspace_id** - (*int*): Numerical of the id the model is in.
  * **name** - (*string*): Name of the model.
  * **updated_at** - (*DateTime*): Date and time of the model's last update.
  * **created_at** - (*DateTime*): Date and time of the model's creation.
  * **model_config** - (*string*): Details of the model's configuration.
  
Example:  Retrieve the details for the model uploaded in the Upload Model to Workspace step.

In [52]:
# Get model details by id

apiRequest = "/models/get_by_id"

data = {
  "id": exampleModelId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'id': 11,
 'owner_id': '""',
 'workspace_id': 5,
 'name': 'apitestmodel-08b16b74-837c-4b4b-a6b7-49475ddece98',
 'updated_at': '2022-11-28T21:30:05.071826+00:00',
 'created_at': '2022-11-28T21:30:05.071826+00:00',
 'model_config': None}

### Get Model Versions

Retrieves all versions of a model based on either the name of the model or the `model_pk_id`.

* **Parameters**
  * **model_id** - (*REQUIRED String*): The model name.
  * **models_pk_id** - (*REQUIRED int*): The model integer pk id.
* **Returns**
  * Array(Model Details)
    * **sha** - (*String*): The `sha` hash of the model version.
    * **models_pk_id**- (*int*): The pk id of the model.
    * **model_version** - (*String*): The UUID identifier of the model version.
    * **owner_id** - (*String*): The Keycloak user id of the model's owner.
    * **model_id** - (*String*): The name of the model.
    * **id** - (*int*): The integer id of the model.
    * **file_name** - (*String*): The filename used when uploading the model.
    * **image_path** - (*String*): The image path of the model.


Example:  Retrieve the versions for a previously uploaded model. The variables `exampleModelVersion` and `exampleModelSha` will store the model's version and SHA values for use in other examples.

In [32]:
# List models in a workspace

apiRequest = "/models/list_versions"

data = {
  "model_id": exampleModelName,
  "models_pk_id": exampleModelId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

[{'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
  'models_pk_id': 68,
  'model_version': '87476d85-b8ee-4714-81ba-53041b26f50f',
  'owner_id': '""',
  'model_id': 'apitestmodel-dfa7e9fd-df72-4b28-93f6-3d147c9f962f',
  'id': 68,
  'file_name': 'ccfraud.onnx',
  'image_path': None}]

In [33]:
# Stored for future examples

exampleModelVersion = response[0]['model_version']
exampleModelSha = response[0]['sha']

### Get Model Configuration by Id

Returns the model's configuration details.

* **Parameters**
  * **model_id** - (*REQUIRED int*): The numerical value of the model's id.
  
Example:  Submit the model id for the model uploaded in the Upload Model to Workspace step to retrieve configuration details.

In [55]:
# Get model config by id

apiRequest = "/models/get_config_by_id"

data = {
  "model_id": exampleModelId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'model_config': None}

### Get Model Details

Returns details regarding a single model, including versions.

Returns the model's configuration details.

* **Parameters**
  * **model_id** - (*REQUIRED int*): The numerical value of the model's id.
  
Example:  Submit the model id for the model uploaded in the Upload Model to Workspace step to retrieve configuration details.

In [56]:
# Get model config by id

apiRequest = "/models/get"

data = {
  "id": exampleModelId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'id': 11,
 'name': 'apitestmodel-08b16b74-837c-4b4b-a6b7-49475ddece98',
 'owner_id': '""',
 'created_at': '2022-11-28T21:30:05.071826+00:00',
 'updated_at': '2022-11-28T21:30:05.071826+00:00',
 'models': [{'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
   'models_pk_id': 11,
   'model_version': '2589d7e4-bc61-4cb7-8278-5399f28ad001',
   'owner_id': '""',
   'model_id': 'apitestmodel-08b16b74-837c-4b4b-a6b7-49475ddece98',
   'id': 11,
   'file_name': 'streamfile.onnx',
   'image_path': None}]}

## Pipeline Management

Pipelines can be managed through the Wallaroo API.  Pipelines are the vehicle used for deploying, serving, and monitoring ML models.  For more information, see the [Wallaroo Glossary](https://docs.wallaroo.ai/wallaroo-glossary/).

### Create Pipeline in a Workspace

Creates a new pipeline in the specified workspace.

* **Parameters**
  * **pipeline_id** - (REQUIRED string): Name of the new pipeline.
  * **workspace_id** - (REQUIRED int): Numerical id of the workspace for the new pipeline.
  * **definition** - (REQUIRED string): Pipeline definitions, can be `{}` for none.

Example:  Two pipelines are created in the workspace created in the step Create Workspace.  One will be an empty pipeline without any models, the other will be created using the uploaded models in the Upload Model to Workspace step and no configuration details.  The pipeline id, variant id, and variant version of each pipeline will be stored for later examples.

In [57]:
# Create pipeline in a workspace

apiRequest = "/pipelines/create"

exampleEmptyPipelineName=f"emptypipeline-{uuid.uuid4()}"

data = {
  "pipeline_id": exampleEmptyPipelineName,
  "workspace_id": exampleWorkspaceId,
  "definition": {}
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
exampleEmptyPipelineId = response['pipeline_pk_id']
exampleEmptyPipelineVariantId=response['pipeline_variant_pk_id']
emptyExamplePipelineVariantVersion=['pipeline_variant_version']
response

{'pipeline_pk_id': 3,
 'pipeline_variant_pk_id': 3,
 'pipeline_variant_version': '84730f78-7b89-4420-bdcb-3c5abac0dd10'}

In [34]:
# Create pipeline in a workspace with models

apiRequest = "/pipelines/create"

exampleModelPipelineName=f"pipelinewithmodel-{uuid.uuid4()}"
exampleModelDeployName = f"deploywithmodel-{uuid.uuid4()}"

data = {
  "pipeline_id": exampleModelPipelineName,
  "workspace_id": exampleWorkspaceId,
  "definition": {
      "id":exampleModelDeployName,
      "steps":
      [
          {
          "ModelInference":
          {
              "models": [
                    {
                        "name":exampleModelName,
                        "version":exampleModelVersion,
                        "sha":exampleModelSha
                    }
                ]
          }
          }
      ]
  }
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
exampleModelPipelineId = response['pipeline_pk_id']
exampleModelPipelineVariantId=response['pipeline_variant_pk_id']
emptyModelPipelineVariantVersion=['pipeline_variant_version']
response

{'pipeline_pk_id': 108,
 'pipeline_variant_pk_id': 108,
 'pipeline_variant_version': '246685fd-258a-49a6-b431-ae4847890eee'}

### Deploy a Pipeline

Deploy a an existing pipeline.  Note that for any pipeline that has model steps, they must be included either in `model_configs`, `model_ids` or `models`.

* **Parameters**
  * **deploy_id** (*REQUIRED string*): The name for the pipeline deployment.
  * **engine_config** (*OPTIONAL string*): Additional configuration options for the pipeline.
  * **pipeline_version_pk_id** (*REQUIRED int*): Pipeline version id.
  * **model_configs** (*OPTIONALArray int*): Ids of model configs to apply.
  * **model_ids** (*OPTIONALArray int*): Ids of models to apply to the pipeline.  If passed in, model_configs will be created automatically.
  * **models** (*OPTIONAL Array models*):  If the model ids are not available as a pipeline step, the models' data can be passed to it through this method.  The options below are only required if `models` are provided as a parameter.
    * **name** (*REQUIRED string*): Name of the uploaded model that is in the same workspace as the pipeline.
    * **version** (*REQUIRED string*): Version of the model to use.
    * **sha** (*REQUIRED string*): SHA value of the model.
  * **pipeline_id** (*REQUIRED int*): Numerical value of the pipeline to deploy.
* **Returns**
  * **id** (*int*): The deployment id.

Examples:  Both the empty pipeline and pipeline with model created in the step [Create Pipeline in a Workspace](#create-pipeline-in-a-workspace) will be deployed and their deployment information saved for later examples.


In [59]:
# Deploy empty pipeline

apiRequest = "/pipelines/deploy"

exampleEmptyDeployId = f"emptydeploy-{uuid.uuid4()}"

data = {
    "deploy_id": exampleEmptyDeployId,
    "pipeline_version_pk_id": exampleEmptyPipelineVariantId,
    "pipeline_id": exampleEmptyPipelineId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
exampleEmptyDeploymentId=response['id']
response


{'id': 2}

In [35]:
# Deploy a pipeline with models

apiRequest = "/pipelines/deploy"
exampleModelDeployId=f"modeldeploy-{uuid.uuid4()}"

data = {
    "deploy_id": exampleModelDeployId,
    "pipeline_version_pk_id": exampleModelPipelineVariantId,
    "models": [
        {
            "name":exampleModelName,
            "version":exampleModelVersion,
            "sha":exampleModelSha
        }
    ],
    "pipeline_id": exampleModelPipelineId
}


response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
exampleModelDeploymentId=response['id']
response

{'id': 60}

### Get Deployment Status

Returns the deployment status.

* **Parameters**
  * **name** - (REQUIRED string): The deployment in the format {deployment_name}-{deploymnent-id}.
  
Example: The deployed empty and model pipelines status will be displayed.

In [ ]:
# Get empty pipeline deployment

apiRequest = "/status/get_deployment"

data = {
  "name": f"{exampleEmptyDeployId}-{exampleEmptyDeploymentId}"
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

In [40]:
# Get model pipeline deployment

apiRequest = "/status/get_deployment"

data = {
  "name": f"{exampleModelDeployId}-{exampleModelDeploymentId}"
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.4.1.151',
   'name': 'engine-577db84597-x7bm4',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'pipelinewithmodel-94676967-b018-4002-89ef-1d69defc6273',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'apitestmodel-dfa7e9fd-df72-4b28-93f6-3d147c9f962f',
      'version': '87476d85-b8ee-4714-81ba-53041b26f50f',
      'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.4.2.59',
   'name': 'engine-lb-7d6f4bfdd-6hxj5',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Get External Inference URL

The API command `/admin/get_pipeline_external_url` retrieves the external inference URL for a specific pipeline in a workspace.

* **Parameters**
  * **workspace_id** (*REQUIRED integer*):  The workspace integer id.
  * **pipeline_name** (*REQUIRED string*): The name of the pipeline.

In this example, a list of the workspaces will be retrieved.  Based on the setup from the Internal Pipeline Deployment URL Tutorial, the workspace matching `urlworkspace` will have it's **workspace id** stored and used for the `/admin/get_pipeline_external_url` request with the pipeline `urlpipeline`.

The External Inference URL will be stored as a variable for the next step.

In [ ]:
## Retrieve the pipeline's External Inference URL

apiRequest = "/admin/get_pipeline_external_url"


data = {
    "workspace_id": exampleWorkspaceId,
    "pipeline_name": exampleModelPipelineName
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
print(response)
externalUrl = response['url']
externalUrl

In [ ]:
exampleModelPipelineName

### Perform Inference Through External URL

The inference can now be performed through the External Inference URL.  This URL will accept the same inference data file that is used with the Wallaroo SDK, or with an Internal Inference URL as used in the Internal Pipeline Inference URL Tutorial.

For this example, the `externalUrl` retrieved through the [Get External Inference URL](#get-external-inference-url) is used to submit a single inference request through the data file `data-1.json`.

In [23]:
TOKEN2 = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEaXlQVjFQMUxHdV81WkpseXR3X2RZSnZwUjdodE9EN3V5cGxocjhacHNrIn0.eyJleHAiOjE2NzE1NjY5NDEsImlhdCI6MTY3MTU2MzM0MSwianRpIjoiN2Y2MTgyZmQtYTIyMS00Y2ZlLThjZDUtN2Y1ZDM5Y2MwZTAwIiwiaXNzIjoiaHR0cHM6Ly9hcGl0ZXN0LmtleWNsb2FrLndhbGxhcm9vY29tbXVuaXR5Lm5pbmphL2F1dGgvcmVhbG1zL21hc3RlciIsImF1ZCI6WyJtYXN0ZXItcmVhbG0iLCJhY2NvdW50Il0sInN1YiI6IjEyMmI2MGE1LWRkYzctNDNjOS05ZmM2LTIwZTljMmI3ZDg1NCIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFwaS1jbGllbnQiLCJzZXNzaW9uX3N0YXRlIjoiNGY4ZDk0NDYtZmE1Ny00MGM3LTk1ZDAtM2E1OGY0YzVlMTQxIiwiYWNyIjoiMSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJkZWZhdWx0LXJvbGVzLW1hc3RlciIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJtYXN0ZXItcmVhbG0iOnsicm9sZXMiOlsibWFuYWdlLXVzZXJzIiwidmlldy11c2VycyIsInF1ZXJ5LWdyb3VwcyIsInF1ZXJ5LXVzZXJzIl19LCJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6InByb2ZpbGUgZW1haWwiLCJzaWQiOiI0ZjhkOTQ0Ni1mYTU3LTQwYzctOTVkMC0zYTU4ZjRjNWUxNDEiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiaHR0cHM6Ly9oYXN1cmEuaW8vand0L2NsYWltcyI6eyJ4LWhhc3VyYS11c2VyLWlkIjoiMTIyYjYwYTUtZGRjNy00M2M5LTlmYzYtMjBlOWMyYjdkODU0IiwieC1oYXN1cmEtZGVmYXVsdC1yb2xlIjoidXNlciIsIngtaGFzdXJhLWFsbG93ZWQtcm9sZXMiOlsidXNlciJdLCJ4LWhhc3VyYS11c2VyLWdyb3VwcyI6Int9In0sInByZWZlcnJlZF91c2VybmFtZSI6InNjb3R0IiwiZW1haWwiOiJzY290dEB3YWxsYXJvby5haSJ9.jNw6LJrimK9HxCENWPkKQPVQXMmk2nGP_VGxSUv9vMdxuzYRONLRaTqrXoxz380IlPQT_QGoiMtA0vjMk40-jg0dAmTcWlDA0sGQyKQq2qIctutGCTbiVhOdmc_69kYyePXlYk3fvl16_L_ZcuhGjlS1Gwl1FtrI6jNAjCrBzTQf1MHHZPa77gxZrqK8D95EGYs34WDu359i5S-stPmH0D9OSTFad0UMZvmuTql4YBOIwWpDzgtU8nC_6XLkWYnzlasABDsfsLMggyWntULHeNxDd9wWK0QSdvGedRZdMhy8gRd0DrV3SFTwF9YvlR2q4t6zNbxWXDR-iF8I7Vog4w"

externalURL2 = "https://apitest.api.YOUR SUFFIX/v1/api/pipelines/infer/test1-6"

import json
## Inference through external URL

# retrieve the json data to submit
data = json.load(open('./data/cc_data_1k.json','rb'))

# set the headers
headers= {
        'Authorization': 'Bearer ' + TOKEN2
    }

# submit the request via POST
response = requests.post(externalURL2, json=data, headers=headers)

# Only the first 300 characters will be displayed for brevity
printResponse = json.dumps(response.json())
print(printResponse[0:300])


[{"model_name": "apitestmodel-418716fb-f3da-4e11-bde5-be323c9382fe", "model_version": "1e794d4e-21d1-4140-9a52-7a5252094e7b", "pipeline_name": "test1", "outputs": [{"Float": {"v": 1, "dim": [1001, 1], "data": [0.993003249168396, 0.993003249168396, 0.993003249168396, 0.993003249168396, 0.001091688871


### Undeploy a Pipeline

Undeploys a deployed pipeline.

* **Parameters**
  * **pipeline_id** - (*REQUIRED int*): The numerical id of the pipeline.
  * **deployment_id** - (*REQUIRED int*): The numerical id of the deployment.
* **Returns**
  * Nothing if the call is successful.

Example:  Both the empty pipeline and pipeline with models deployed in the step Deploy a Pipeline will be undeployed.

In [63]:
# Undeploy an empty pipeline

apiRequest = "/pipelines/undeploy"

data = {
    "pipeline_id": exampleEmptyPipelineId,
    "deployment_id":exampleEmptyDeploymentId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

In [64]:
# Undeploy pipeline with models

apiRequest = "/pipelines/undeploy"

data = {
    "pipeline_id": exampleModelPipelineId,
    "deployment_id":exampleModelDeploymentId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

### Copy a Pipeline

Copies an existing pipeline into a new one in the same workspace.  A new engine configuration can be set for the copied pipeline.

* **Parameters**
  * **name** - (REQUIRED string): The name of the new pipeline.
  * **workspace_id** - (REQUIRED int): The numerical id of the workspace to copy the source pipeline from.
  * **source_pipeline** - (REQUIRED int): The numerical id of the pipeline to copy from.
  * **deploy** - (OPTIONAL string): Name of the deployment.
  * **engine_config** - (OPTIONAL string): Engine configuration options.
  * **pipeline_version** - (OPTIONAL string): Optional version of the copied pipeline to create.

Example:  The pipeline with models created in the step Create Pipeline in a Workspace will be copied into a new one.


In [65]:
# Copy a pipeline

apiRequest = "/pipelines/copy"

exampleCopiedPipelineName=f"copiedmodelpipeline-{uuid.uuid4()}"

data = {
  "name": exampleCopiedPipelineName,
  "workspace_id": exampleWorkspaceId,
  "source_pipeline": exampleModelPipelineId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'pipeline_pk_id': 5,
 'pipeline_variant_pk_id': 5,
 'pipeline_version': None,
 'deployment': None}

## List Enablement Features

Lists the enablement features for the Wallaroo instance.

* **PARAMETERS**
  * null:  An empty set `{}`
* **RETURNS**
  * **features** - (*string*): Enabled features.
  * **name** - (*string*): Name of the Wallaroo instance.
  * **is_auth_enabled** - (*bool*): Whether authentication is enabled.

In [66]:
# List enablement features

apiRequest = "/features/list"

data = {
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'features': {'plateau': 'true'},
 'name': 'Wallaroo Dev',
 'is_auth_enabled': True}

## Assays

**IMPORTANT NOTE**: These assays were run in a Wallaroo environment with canned historical data.  See the [Wallaroo Assay Tutorial](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_insights) for details on setting up this environment.  This historical data is **required** for these examples.

### Create Assay

Create a new array in a specified pipeline.

* **PARAMETERS**
  * **id** - (*OPTIONAL int*):  The numerical identifier for the assay.
  * **name** - (*REQUIRED string*): The name of the assay.
  * **pipeline_id** - (*REQUIRED int*): The numerical idenfifier the assay will be placed into.
  * **pipeline_name** - (*REQUIRED string*): The name of the pipeline
  * **active** - (*REQUIRED bool*): Indicates whether the assay will be active upon creation or not.
  * **status** - (*REQUIRED string*): The status of the assay upon creation.
  * **iopath** - (*REQUIRED string*): The iopath of the assay.
  * **baseline** - (*REQUIRED baseline*): The baseline for the assay.
    * **Fixed** - (*REQUIRED AssayFixConfiguration*): The fixed configuration for the assay.
      * **pipeline** - (*REQUIRED string*): The name of the pipeline with the baseline data.
      * **model** - (*REQUIRED string*): The name of the model used.
      * **start_at** - (*REQUIRED string*): The DateTime of the baseline start date.
      * **end_at** - (*REQUIRED string*): The DateTime of the baseline end date.
  * **window** (*REQUIRED AssayWindow*): Assay window.
    * **pipeline** - (*REQUIRED string*): The name of the pipeline for the assay window.
    * **model** - (*REQUIRED string*): The name of the model used for the assay window.
    * **width** - (*REQUIRED string*): The width of the assay window.
    * **start** - (*OPTIONAL string*): The DateTime of when to start the assay window.
    * **interval** - (*OPTIONAL string*): The assay window interval.
  * **summarizer** - (*REQUIRED AssaySummerizer*): The summarizer type for the array aka "advanced settings" in the Wallaroo Dashboard UI.
    * **type** - (*REQUIRED string*): Type of summarizer.
    * **bin_mode** - (*REQUIRED string*): The binning model type.  Values can be:
      * Quantile
      * Equal
    * **aggregation** - (*REQUIRED string*): Aggregation type.
    * **metric** - (*REQUIRED string*): Metric type.  Values can be:
      * PSI
      * Maximum Difference of Bins
      * Sum of the Difference of Bins
    * **num_bins** - (*REQUIRED int*): The number of bins.  Recommanded values are between 5 and 14.
    * **bin_weights** - (*OPTIONAL AssayBinWeight*): The weights assigned to the assay bins.
    * **bin_width** - (*OPTIONAL AssayBinWidth*): The width assigned to the assay bins.
    * **provided_edges** - (*OPTIONAL AssayProvidedEdges*): The edges used for the assay bins.
    * **add_outlier_edges** - (*REQUIRED bool*): Indicates whether to add outlier edges or not.
  * **warning_threshold** - (*OPTIONAL number*): Optional warning threshold.
  * **alert_threshold** - (*REQUIRED number*): Alert threshold.
  * **run_until** - (*OPTIONAL string*): DateTime of when to end the assay.
  * **workspace_id** - (*REQUIRED integer*): The workspace the assay is part of.
  * **model_insights_url** - (*OPTIONAL string*): URL for model insights.
* **RETURNS**
  * **assay_id** - (*integer*): The id of the new assay.

As noted this example requires the [Wallaroo Assay Tutorial](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_insights) for historical data.  Before running this example, set the sample pipeline id, pipeline, name, model name, and workspace id in the code sample below.  For more information on retrieving this information, see the [Wallaroo Developer Guides](https://docs.wallaroo.ai/wallaroo-developer-guides/).

In [51]:
# Create assay

apiRequest = "/assays/create"

exampleAssayName = "api_assay_test2"

## Now get all of the assays for the pipeline in workspace 4 `housepricedrift`

exampleAssayPipelineId = 4
exampleAssayPipelineName = "housepricepipe"
exampleAssayModelName = "housepricemodel"
exampleAssayWorkspaceId = 4

# iopath can be input 00 or output 0 0
data = {
    'name': exampleAssayName,
    'pipeline_id': exampleAssayPipelineId,
    'pipeline_name': exampleAssayPipelineName,
    'active': True,
    'status': 'active',
    'iopath': "input 0 0",
    'baseline': {
        'Fixed': {
            'pipeline': exampleAssayPipelineName,
            'model': 'houseprice-model-yns',
            'start_at': '2022-01-01T00:00:00-05:00',
            'end_at': '2022-01-02T00:00:00-05:00'
        }
    },
    'window': {
        'pipeline': exampleAssayPipelineName,
        'model': exampleAssayModelName,
        'width': '24 hours',
        'start': None,
        'interval': None
    },
    'summarizer': {
        'type': 'UnivariateContinuous',
        'bin_mode': 'Quantile',
        'aggregation': 'Density',
        'metric': 'PSI',
        'num_bins': 5,
        'bin_weights': None,
        'bin_width': None,
        'provided_edges': None,
        'add_outlier_edges': True
    },
    'warning_threshold': 0,
    'alert_threshold': 0.1,
    'run_until': None,
    'workspace_id': exampleAssayWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
example_assay_id = response['assay_id']
response

{'assay_id': 2}

### List Assays

Lists all assays in the specified pipeline.

* **PARAMETERS**
  * **pipeline_id** - (*REQUIRED int*):  The numerical ID of the pipeline.
* **RETURNS**
  * **assays** - (*Array assays*): A list of all assays.

Example:  Display a list of all assays in a workspace.  This will assume we have a workspace with an existing Assay and the associated data has been upload.  See the tutorial [Wallaroo Assays Tutorial](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_insights).

For this reason, these values are hard coded for now.

In [62]:
## First list all of the workspaces and the list of pipelines

# List workspaces

apiRequest = "/workspaces/list"

data = {
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'workspaces': [{'id': 1,
   'name': 'john.hansarick@wallaroo.ai - Default Workspace',
   'created_at': '2022-10-10T16:32:45.355874+00:00',
   'created_by': 'f68760ad-a27c-4f9b-808f-0b512f07571f',
   'archived': False,
   'models': [],
   'pipelines': []},
  {'id': 3,
   'name': 'testapiworkspace-e87e543f-25f1-4f6d-82c6-4eb48902575a',
   'created_at': '2022-10-10T18:25:27.926919+00:00',
   'created_by': 'f68760ad-a27c-4f9b-808f-0b512f07571f',
   'archived': False,
   'models': [1],
   'pipelines': [1, 2, 3]},
  {'id': 4,
   'name': 'housepricedrift',
   'created_at': '2022-10-10T18:38:50.748057+00:00',
   'created_by': 'f68760ad-a27c-4f9b-808f-0b512f07571f',
   'archived': False,
   'models': [2],
   'pipelines': [4]},
  {'id': 5,
   'name': 'housepricedrifts',
   'created_at': '2022-10-10T18:45:00.152716+00:00',
   'created_by': 'f68760ad-a27c-4f9b-808f-0b512f07571f',
   'archived': False,
   'models': [],
   'pipelines': []}]}

In [83]:
# Get assays

apiRequest = "/assays/list"

data = {
    "pipeline_id": exampleAssayPipelineId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

[{'id': 3,
  'name': 'example assay',
  'active': True,
  'status': 'created',
  'warning_threshold': None,
  'alert_threshold': 0.1,
  'pipeline_id': 4,
  'pipeline_name': 'housepricepipe',
  'last_run': None,
  'next_run': '2022-10-10T19:00:43.941894+00:00',
  'run_until': None,
  'updated_at': '2022-10-10T19:00:43.945411+00:00',
  'baseline': {'Fixed': {'pipeline': 'housepricepipe',
    'model': 'housepricemodel',
    'start_at': '2022-01-01T00:00:00+00:00',
    'end_at': '2022-01-02T00:00:00+00:00'}},
  'window': {'pipeline': 'housepricepipe',
   'model': 'housepricemodel',
   'width': '24 hours',
   'start': None,
   'interval': None},
  'summarizer': {'type': 'UnivariateContinuous',
   'bin_mode': 'Quantile',
   'aggregation': 'Density',
   'metric': 'PSI',
   'num_bins': 5,
   'bin_weights': None,
   'bin_width': None,
   'provided_edges': None,
   'add_outlier_edges': True}},
 {'id': 2,
  'name': 'api_assay_test2',
  'active': True,
  'status': 'created',
  'warning_threshold':

## Activate or Deactivate Assay

Activates or deactivates an existing assay.

* **Parameters**
  * **id** - (*REQUIRED int*): The numerical id of the assay.
  * **active** - (*REQUIRED bool*): True to activate the assay, False to deactivate it.
* **Returns**
  * * **id** - (*integer*): The numerical id of the assay.
  * **active** - (*bool*): True to activate the assay, False to deactivate it.

Example:  Assay 8 "House Output Assay" will be deactivated then activated.

In [54]:
# Deactivate assay

apiRequest = "/assays/set_active"

data = {
    'id': example_assay_id,
    'active': False
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'id': 2, 'active': False}

In [66]:
# Activate assay

apiRequest = "/assays/set_active"

data = {
    'id': example_assay_id,
    'active': True
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'id': 2, 'active': True}

### Create Interactive Baseline

Creates an interactive assay baseline.

* **PARAMETERS**
  * **id** - (*REQUIRED int*):  The numerical identifier for the assay.
  * **name** - (*REQUIRED string*): The name of the assay.
  * **pipeline_id** - (*REQUIRED int*): The numerical idenfifier the assay will be placed into.
  * **pipeline_name** - (*REQUIRED string*): The name of the pipeline
  * **active** - (*REQUIRED bool*): Indicates whether the assay will be active upon creation or not.
  * **status** - (*REQUIRED string*): The status of the assay upon creation.
  * **iopath** - (*REQUIRED string*): The iopath of the assay.
  * **baseline** - (*REQUIRED baseline*): The baseline for the assay.
    * **Fixed** - (*REQUIRED AssayFixConfiguration*): The fixed configuration for the assay.
      * **pipeline** - (*REQUIRED string*): The name of the pipeline with the baseline data.
      * **model** - (*REQUIRED string*): The name of the model used.
      * **start_at** - (*REQUIRED string*): The DateTime of the baseline start date.
      * **end_at** - (*REQUIRED string*): The DateTime of the baseline end date.
  * **window** (*REQUIRED AssayWindow*): Assay window.
    * **pipeline** - (*REQUIRED string*): The name of the pipeline for the assay window.
    * **model** - (*REQUIRED string*): The name of the model used for the assay window.
    * **width** - (*REQUIRED string*): The width of the assay window.
    * **start** - (*OPTIONAL string*): The DateTime of when to start the assay window.
    * **interval** - (*OPTIONAL string*): The assay window interval.
  * **summarizer** - (*REQUIRED AssaySummerizer*): The summarizer type for the array aka "advanced settings" in the Wallaroo Dashboard UI.
    * **type** - (*REQUIRED string*): Type of summarizer.
    * **bin_mode** - (*REQUIRED string*): The binning model type.  Values can be:
      * Quantile
      * Equal
    * **aggregation** - (*REQUIRED string*): Aggregation type.
    * **metric** - (*REQUIRED string*): Metric type.  Values can be:
      * PSI
      * Maximum Difference of Bins
      * Sum of the Difference of Bins
    * **num_bins** - (*REQUIRED int*): The number of bins.  Recommanded values are between 5 and 14.
    * **bin_weights** - (*OPTIONAL AssayBinWeight*): The weights assigned to the assay bins.
    * **bin_width** - (*OPTIONAL AssayBinWidth*): The width assigned to the assay bins.
    * **provided_edges** - (*OPTIONAL AssayProvidedEdges*): The edges used for the assay bins.
    * **add_outlier_edges** - (*REQUIRED bool*): Indicates whether to add outlier edges or not.
  * **warning_threshold** - (*OPTIONAL number*): Optional warning threshold.
  * **alert_threshold** - (*REQUIRED number*): Alert threshold.
  * **run_until** - (*OPTIONAL string*): DateTime of when to end the assay.
  * **workspace_id** - (*REQUIRED integer*): The workspace the assay is part of.
  * **model_insights_url** - (*OPTIONAL string*): URL for model insights.
* **RETURNS**
  * {} when successful.

Example:  An interactive assay baseline will be set for the assay "Test Assay" on Pipeline 4.

In [69]:
# Run interactive baseline

apiRequest = "/assays/run_interactive_baseline"

exampleAssayPipelineId = 4
exampleAssayPipelineName = "housepricepipe"
exampleAssayModelName = "housepricemodel"
exampleAssayWorkspaceId = 4
exampleAssayId = 3
exampleAssayName = "example assay"

data = {
    'id': exampleAssayId,
    'name': exampleAssayName,
    'pipeline_id': exampleAssayPipelineId,
    'pipeline_name': exampleAssayPipelineName,
    'active': True,
    'status': 'active',
    'iopath': "input 0 0",
    'baseline': {
        'Fixed': {
            'pipeline': exampleAssayPipelineName,
            'model': exampleAssayModelName,
            'start_at': '2022-01-01T00:00:00-05:00',
            'end_at': '2022-01-02T00:00:00-05:00'
        }
    },
    'window': {
        'pipeline': exampleAssayPipelineName,
        'model': exampleAssayModelName,
        'width': '24 hours',
        'start': None,
        'interval': None
    },
    'summarizer': {
        'type': 'UnivariateContinuous',
        'bin_mode': 'Quantile',
        'aggregation': 'Density',
        'metric': 'PSI',
        'num_bins': 5,
        'bin_weights': None,
        'bin_width': None,
        'provided_edges': None,
        'add_outlier_edges': True
    },
    'warning_threshold': 0,
    'alert_threshold': 0.1,
    'run_until': None,
    'workspace_id': exampleAssayWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'assay_id': 3,
 'name': 'example assay',
 'created_at': 1665428974654,
 'elapsed_millis': 3,
 'pipeline_id': 4,
 'pipeline_name': 'housepricepipe',
 'iopath': 'input 0 0',
 'baseline_summary': {'count': 1812,
  'min': -3.6163810435665233,
  'max': 7.112734553641073,
  'mean': 0.03518936967736047,
  'median': -0.39764636440424433,
  'std': 0.9885006118746916,
  'edges': [-3.6163810435665233,
   -0.39764636440424433,
   -0.39764636440424433,
   0.6752651953165153,
   0.6752651953165153,
   7.112734553641073,
   None],
  'edge_names': ['left_outlier',
   'q_20',
   'q_40',
   'q_60',
   'q_80',
   'q_100',
   'right_outlier'],
  'aggregated_values': [0.0,
   0.5739514348785872,
   0.0,
   0.3383002207505519,
   0.0,
   0.08774834437086093,
   0.0],
  'aggregation': 'Density',
  'start': '2022-01-01T05:00:00Z',
  'end': '2022-01-02T05:00:00Z'},
 'window_summary': {'count': 1812,
  'min': -3.6163810435665233,
  'max': 7.112734553641073,
  'mean': 0.03518936967736047,
  'median': -0.3976463

### Get Assay Baseline

Retrieve an assay baseline.

* **Parameters**
  * **workspace_id** - (*REQUIRED integer*): Numerical id for the workspace the assay is in.
  * **pipeline_name** - (*REQUIRED string*): Name of the pipeline the assay is in.
  * **start** - (*OPTIONAL string*): DateTime for when the baseline starts.
  * **end** - (*OPTIONAL string*): DateTime for when the baseline ends.
  * **model_name** - (*OPTIONAL string*): Name of the model.
  * **limit** - (*OPTIONAL integer*): Maximum number of baselines to return.
* **Returns**
  * Assay Baseline
  
Example:  3 assay baselines for Workspace 6 and pipeline `houseprice-pipe-yns` will be retrieved.

In [84]:
# Get Assay Baseline

apiRequest = "/assays/get_baseline"

data = {
    'workspace_id': exampleAssayWorkspaceId,
    'pipeline_name': exampleAssayPipelineName,
    'limit': 3
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

[{'check_failures': [],
  'elapsed': 138,
  'model_name': 'housepricemodel',
  'model_version': 'test_version',
  'original_data': {'tensor': [[0.6752651953165153,
     0.4999342471069234,
     -0.1508359058521761,
     0.20024994573167013,
     -0.08666382440547035,
     0.009116407905326388,
     -0.002872821251696453,
     -0.9179715198382244,
     -0.305653139057544,
     2.4393894526979074,
     0.29288456205300767,
     -0.3485179782510063,
     1.1121054807107582,
     0.20193559456886756,
     -0.20817781526102327,
     1.0279052268485522,
     -0.0196096612880121]]},
  'outputs': [{'Float': {'data': [13.262725830078123],
     'dim': [1, 1],
     'v': 1}}],
  'pipeline_name': 'housepricepipe',
  'time': 1643673456974},
 {'check_failures': [],
  'elapsed': 136,
  'model_name': 'housepricemodel',
  'model_version': 'test_version',
  'original_data': {'tensor': [[-0.39764636440424433,
     -1.4463372267359147,
     0.044822346031326635,
     -0.4259897870655369,
     -0.0866638244

### Run Assay Interactively

Runs an assay.

* **Parameters**
  * **id** - (*REQUIRED int*):  The numerical identifier for the assay.
  * **name** - (*REQUIRED string*): The name of the assay.
  * **pipeline_id** - (*REQUIRED int*): The numerical idenfifier the assay will be placed into.
  * **pipeline_name** - (*REQUIRED string*): The name of the pipeline
  * **active** - (*REQUIRED bool*): Indicates whether the assay will be active upon creation or not.
  * **status** - (*REQUIRED string*): The status of the assay upon creation.
  * **iopath** - (*REQUIRED string*): The iopath of the assay.
  * **baseline** - (*REQUIRED baseline*): The baseline for the assay.
    * **Fixed** - (*REQUIRED AssayFixConfiguration*): The fixed configuration for the assay.
      * **pipeline** - (*REQUIRED string*): The name of the pipeline with the baseline data.
      * **model** - (*REQUIRED string*): The name of the model used.
      * **start_at** - (*REQUIRED string*): The DateTime of the baseline start date.
      * **end_at** - (*REQUIRED string*): The DateTime of the baseline end date.
  * **window** (*REQUIRED AssayWindow*): Assay window.
    * **pipeline** - (*REQUIRED string*): The name of the pipeline for the assay window.
    * **model** - (*REQUIRED string*): The name of the model used for the assay window.
    * **width** - (*REQUIRED string*): The width of the assay window.
    * **start** - (*OPTIONAL string*): The DateTime of when to start the assay window.
    * **interval** - (*OPTIONAL string*): The assay window interval.
  * **summarizer** - (*REQUIRED AssaySummerizer*): The summarizer type for the array aka "advanced settings" in the Wallaroo Dashboard UI.
    * **type** - (*REQUIRED string*): Type of summarizer.
    * **bin_mode** - (*REQUIRED string*): The binning model type.  Values can be:
      * Quantile
      * Equal
    * **aggregation** - (*REQUIRED string*): Aggregation type.
    * **metric** - (*REQUIRED string*): Metric type.  Values can be:
      * PSI
      * Maximum Difference of Bins
      * Sum of the Difference of Bins
    * **num_bins** - (*REQUIRED int*): The number of bins.  Recommanded values are between 5 and 14.
    * **bin_weights** - (*OPTIONAL AssayBinWeight*): The weights assigned to the assay bins.
    * **bin_width** - (*OPTIONAL AssayBinWidth*): The width assigned to the assay bins.
    * **provided_edges** - (*OPTIONAL AssayProvidedEdges*): The edges used for the assay bins.
    * **add_outlier_edges** - (*REQUIRED bool*): Indicates whether to add outlier edges or not.
  * **warning_threshold** - (*OPTIONAL number*): Optional warning threshold.
  * **alert_threshold** - (*REQUIRED number*): Alert threshold.
  * **run_until** - (*OPTIONAL string*): DateTime of when to end the assay.
  * **workspace_id** - (*REQUIRED integer*): The workspace the assay is part of.
  * **model_insights_url** - (*OPTIONAL string*): URL for model insights.
* **Returns**
  * Assay
  
Example:  An interactive assay will be run for Assay exampleAssayId exampleAssayName.  Depending on the number of assay results and the data window, this may take some time.  This returns *all* of the results for this assay at this time.  The total number of responses will be displayed after.

In [85]:
# Run interactive assay

apiRequest = "/assays/run_interactive"

data = {
    'id': exampleAssayId,
    'name': exampleAssayName,
    'pipeline_id': exampleAssayPipelineId,
    'pipeline_name': exampleAssayPipelineName,
    'active': True,
    'status': 'active',
    'iopath': "input 0 0",
    'baseline': {
        'Fixed': {
            'pipeline': exampleAssayPipelineName,
            'model': exampleAssayModelName,
            'start_at': '2022-01-01T00:00:00-05:00',
            'end_at': '2022-01-02T00:00:00-05:00'
        }
    },
    'window': {
        'pipeline': exampleAssayPipelineName,
        'model': exampleAssayModelName,
        'width': '24 hours',
        'start': None,
        'interval': None
    },
    'summarizer': {
        'type': 'UnivariateContinuous',
        'bin_mode': 'Quantile',
        'aggregation': 'Density',
        'metric': 'PSI',
        'num_bins': 5,
        'bin_weights': None,
        'bin_width': None,
        'provided_edges': None,
        'add_outlier_edges': True
    },
    'warning_threshold': 0,
    'alert_threshold': 0.1,
    'run_until': None,
    'workspace_id': exampleAssayWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response[0]

{'assay_id': 3,
 'name': 'example assay',
 'created_at': 1665429281268,
 'elapsed_millis': 178,
 'pipeline_id': 4,
 'pipeline_name': 'housepricepipe',
 'iopath': 'input 0 0',
 'baseline_summary': {'count': 1812,
  'min': -3.6163810435665233,
  'max': 7.112734553641073,
  'mean': 0.03518936967736047,
  'median': -0.39764636440424433,
  'std': 0.9885006118746916,
  'edges': [-3.6163810435665233,
   -0.39764636440424433,
   -0.39764636440424433,
   0.6752651953165153,
   0.6752651953165153,
   7.112734553641073,
   None],
  'edge_names': ['left_outlier',
   'q_20',
   'q_40',
   'q_60',
   'q_80',
   'q_100',
   'right_outlier'],
  'aggregated_values': [0.0,
   0.5739514348785872,
   0.0,
   0.3383002207505519,
   0.0,
   0.08774834437086093,
   0.0],
  'aggregation': 'Density',
  'start': '2022-01-01T05:00:00Z',
  'end': '2022-01-02T05:00:00Z'},
 'window_summary': {'count': 1812,
  'min': -3.6163810435665233,
  'max': 3.8939998744787943,
  'mean': 0.006175756859303479,
  'median': -0.397

In [86]:
print(len(response))

30


### Get Assay Results

Retrieve the results for an assay.

* **Parameters**
  * **assay_id** - (*REQUIRED integer*): Numerical id for the assay.
  * **start** - (*OPTIONAL string*): DateTime for when the baseline starts.
  * **end** - (*OPTIONAL string*): DateTime for when the baseline ends.
  * **limit** - (*OPTIONAL integer*): Maximum number of results to return.
  * **pipeline_id** - (*OPTIONAL integer*): Numerical id of the pipeline the assay is in.
* **Returns**
  * Assay Baseline
  
Example:  Results for Assay 3 "example assay" will be retrieved for January 2 to January 3.  For the sake of time, only the first record will be displayed.

In [ ]:
# Get Assay Results

apiRequest = "/assays/get_results"

data = {
    'assay_id': exampleAssayId,
    'pipeline_id': exampleAssayPipelineId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response